In [ ]:
import os
import pandas as pd
import shap
import joblib
import lightgbm as lgb
from sklearn.inspection import PartialDependenceDisplay
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.inspection import permutation_importance
import numpy as np


## Setting fore reforecast

In [ ]:


best_model_tuples = [
    ('svr', 'svr'),
    ('lgb', 'svr'),
    ('mlr', 'svr'),
    ('mlr', 'svr'),
    ('mlr', 'svr'),
    ('mlr', 'svr'),
    ('lgb', 'svr')
]

features = ['Total solar irradiance (W/m2)',
       'Air temperature  (°C) ', 'Atmosphere (hpa)',
       'Power (MW)', 'hour_sin', 'hour_cos', 'month_sin',
       'month_cos', 'same_time_mean', 'same_time_std', 'same_time_max',
       'same_time_min', 'same_weekday_mean', 'same_weekday_std',
       'same_weekday_max', 'same_weekday_min', 'overall_mean', 'overall_std',
       'overall_max', 'overall_min', 'lag_1', 'lag_2', 'lag_3', 'lag_4',
       'lag_5', 'lag_6', 'lag_7', 'lag_8', 'diff_15min', 'diff_30min',
       'diff_45min', 'diff_60min', 'diff_75min', 'diff_90min', 'diff_105min']


test_size = 2/24


shap_values_list = []


## Reforecast test

In [ ]:
from sklearn.preprocessing import MinMaxScaler


# reforecast shap
for i, df_index in enumerate(a):
  print('----------------------------------------------------------------------------------')
  forecast_model,reforecast_model = best_model_tuples[i]
  start_str, end_str = operation_hours_array[i]

  file = [os.path.join(path+'/result_of_paper/22mon_reforecast_data/', file) for file in os.listdir(path+'/result_of_paper/22mon_reforecast_data/') if file.endswith('.csv') and f'{forecast_model}' in file and f'{str(df_index)}' in file][0]
  print(file)
  df = pd.read_csv(file)

  forecast_model,reforecast_model = best_model_tuples[i]
  start_str, end_str = operation_hours_array[i]
  target = f'error_{forecast_model}'
  shifted_target = f'error_{forecast_model}_shifted'

  df[shifted_target] = df[target].shift(-1)
  df = df.dropna().reset_index(drop=True)

  split_index = int(len(df) * (1 - test_size))

  added_features = features.copy() + [f'pred_{forecast_model}', f'error_{forecast_model}']
  added_features = [col for col in added_features if col in df.columns]
  X = df[added_features]
  y = df[shifted_target]
  X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
  y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
  X_test_scaled = scaler.transform(X_test)
  X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

  # 데이터: X_train, X_test, y_train 필요
  # 모델 불러오기
  reforecast_mlr = joblib.load(f"{path}/result_of_paper/22mon_reforecast_data/mlr_reforecasting_model_{forecast_model}_forecast_{str(df_index)}.joblib")
  reforecast_svr = joblib.load(f"{path}/result_of_paper/22mon_reforecast_data/svr_reforecasting_model_{forecast_model}_forecast_{str(df_index)}.joblib")
  reforecast_lgb_model = lgb.Booster(model_file=f"{path}/result_of_paper/22mon_reforecast_data/lgb_reforecasting_model_{forecast_model}_forecast_{str(df_index)}.txt")

  if reforecast_model == 'mlr':
    # ------------------- MLR -------------------
    explainer_mlr = shap.Explainer(reforecast_mlr, X_train_scaled_df)
    shap_values_mlr = explainer_mlr(X_train_scaled_df)

    shap.summary_plot(shap_values_mlr, X_train_scaled_df)
    shap.plots.bar(shap_values_mlr)

    # ▶ MLR Feature Importance
    mlr_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': reforecast_mlr.coef_
    }).sort_values('importance', key=abs, ascending=False)

    print("\n[MLR Feature Importance]")
    print(mlr_importance)

  elif reforecast_model == 'svr':
    # ------------------- SVR -------------------
    svr_result = permutation_importance(reforecast_svr, X_train_scaled_df, y_train, n_repeats=5,n_jobs=-1, random_state=0)

    svr_importance = pd.DataFrame({
        'feature': X_train_scaled_df.columns,
        'importance': svr_result.importances_mean
    }).sort_values('importance', ascending=False)

    print("\n[SVR Feature Importance (Permutation)]")
    print(svr_importance)

    top_n = 20
    top_features = svr_importance.head(top_n)

    plt.figure(figsize=(8, 6))
    plt.barh(top_features['feature'], top_features['importance'], color='salmon')
    plt.xlabel('Permutation Importance')
    plt.title(f'SVR Feature Importance (Top {top_n})')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

  elif reforecast_model == 'lgb':
  # ------------------- LGB -------------------
    explainer_lgb = shap.Explainer(reforecast_lgb_model)
    shap_values_lgb = explainer_lgb(X_train)

    shap_importance = np.abs(shap_values_lgb.values).mean(axis=0)

    # DataFrame으로 정리
    shap_df = pd.DataFrame({
        'feature': X_test.columns,
        'mean_abs_shap': shap_importance
    }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

    # 출력
    for _, row in shap_df.iterrows():
        print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

    # kneedle 알고리즘 사용
    x = np.arange(len(shap_df))
    y = shap_df['mean_abs_shap'].values
    x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
    y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

    # 직선 시작점과 끝점
    start = np.array([x_scaled[0], y_scaled[0]])
    end = np.array([x_scaled[-1], y_scaled[-1]])

    # 각 점에서 직선까지의 수직 거리 계산
    line_vec = end - start
    line_vec_norm = line_vec / np.linalg.norm(line_vec)
    point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
    projections = np.dot(point_vecs, line_vec_norm)
    projected_points = np.outer(projections, line_vec_norm) + start
    distances = np.linalg.norm(point_vecs - projected_points, axis=1)

    # 가장 멀리 떨어진 점이 elbow point
    elbow_idx = np.argmax(distances)
    elbow_feature = shap_df.loc[elbow_idx, 'feature']
    elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

    # 시각화
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
    plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
    plt.title("SHAP Feature Importance with Elbow Point (Distance Method)")
    plt.xlabel("Feature Rank")
    plt.ylabel("Mean |SHAP value|")
    plt.legend()
    plt.grid(True)
    plt.show()

## Reforecast train for all models

In [ ]:
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from sklearn.utils import resample

def compute_feature_importance(f, X, base_pred, model, n_repeats, rng):
    feature_diffs = []
    for _ in range(n_repeats):
        X_permuted = X.copy()
        X_permuted[f] = rng.permutation(X_permuted[f].values)
        pred_permuted = model.predict(X_permuted.values)
        diff = np.abs(base_pred - pred_permuted)
        feature_diffs.append(diff)
    return f, np.mean(np.stack(feature_diffs), axis=0)

def permutation_importance_per_instance(columns, model, X, n_repeats=5, random_state=42, n_jobs=-1):
    rng = np.random.RandomState(random_state)
    X = X.copy()
    base_pred = model.predict(X.values)
    results = Parallel(n_jobs=n_jobs)(
        delayed(compute_feature_importance)(f, X, base_pred, model, n_repeats, rng)
        for f in columns
    )
    return {f: v for f, v in results}


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import multiprocessing



forecast_models = ['mlr','svr','lgb', 'mlp']
reforecast_models = ['mlr','svr','lgb', 'mlp']

mlr_files = sorted([os.path.join(path+'/result_of_paper/ann/', file) for file in os.listdir(path+'/result_of_paper/ann/') if file.endswith('.csv') and 'mlr' in file])
svr_files = sorted([os.path.join(path+'/result_of_paper/ann/', file) for file in os.listdir(path+'/result_of_paper/ann/') if file.endswith('.csv') and 'svr' in file])
lgb_files = sorted([os.path.join(path+'/result_of_paper/ann/', file) for file in os.listdir(path+'/result_of_paper/ann/') if file.endswith('.csv') and 'lgb' in file])
mlp_files = sorted([os.path.join(path+'/result_of_paper/ann/', file) for file in os.listdir(path+'/result_of_paper/ann/') if file.endswith('.csv') and 'mlp' in file])

mlr_df_list = []
svr_df_list = []
lgb_df_list = []
mlp_df_list = []

for file in mlr_files:
  mlr_df_list.append(pd.read_csv(file))
for file in svr_files:
  svr_df_list.append(pd.read_csv(file))
for file in lgb_files:
  lgb_df_list.append(pd.read_csv(file))
for file in mlp_files:
  mlp_df_list.append(pd.read_csv(file))


test_size = 2/24

df_lists=[mlr_df_list, svr_df_list,lgb_df_list, mlp_df_list]

for forecast_model, df_list in zip(forecast_models, df_lists):
  print('==============================================================================================')
  print(f'forecast_model : {forecast_model.upper()}')

  for df, site in zip(df_list, [1,2,4,5,6,7,8]):
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print(f'site : {site}')
    target = f'error_{forecast_model}'
    shifted_target = f'{target}_shifted'

    for reforecast_model in reforecast_models:
      print('--------------------------------------------------------------------------------------------------------------------------------')

      df_copy = df.copy()
      df_copy[shifted_target] = df_copy[target].shift(-1)
      df_copy = df_copy.dropna().reset_index(drop=True)

      split_index = int(len(df_copy) * (1 - test_size))

      added_features = features.copy() + [f'error_{forecast_model}', f'pred_{forecast_model}']
      added_features = [col for col in added_features if col in df_copy.columns]
      X = df_copy[added_features]
      y = df_copy[shifted_target]
      X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
      y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

      scaler = StandardScaler()
      X_train_scaled = scaler.fit_transform(X_train)
      X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
      X_test_scaled = scaler.transform(X_test)
      X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

      reforecast_mlr = joblib.load(f"{path}/result_of_paper/ann/mlr_reforecasting_model_{forecast_model}_forecast_{str(site)}.joblib")
      reforecast_svr = joblib.load(f"{path}/result_of_paper/ann/svr_reforecasting_model_{forecast_model}_forecast_{str(site)}.joblib")
      reforecast_lgb_model = lgb.Booster(model_file=f"{path}/result_of_paper/ann/lgb_reforecasting_model_{forecast_model}_forecast_{str(site)}.txt")
      reforecast_mlp = joblib.load(f"{path}/result_of_paper/ann/mlp_reforecasting_model_{forecast_model}_forecast_{site}.joblib")


      if reforecast_model == 'mlr':
          # ------------------- MLR -------------------
          explainer_mlr = shap.Explainer(reforecast_mlr, X_train_scaled_df)
          shap_values_mlr = explainer_mlr(X_train_scaled_df)

          shap_importance = np.abs(shap_values_mlr.values).mean(axis=0)

          shap_df = pd.DataFrame({
              'feature': X_train.columns,
              'mean_abs_shap': shap_importance
          }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

          print("\n[MLR Feature Importance (SHAP)]")
          for _, row in shap_df.iterrows():
            print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

          # elbow point
          x = np.arange(len(shap_df))
          y = shap_df['mean_abs_shap'].values
          x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
          y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

          start = np.array([x_scaled[0], y_scaled[0]])
          end = np.array([x_scaled[-1], y_scaled[-1]])

          line_vec = end - start
          line_vec_norm = line_vec / np.linalg.norm(line_vec)
          point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
          projections = np.dot(point_vecs, line_vec_norm)
          projected_points = np.outer(projections, line_vec_norm) + start
          distances = np.linalg.norm(point_vecs - projected_points, axis=1)

          elbow_idx = np.argmax(distances)
          elbow_feature = shap_df.loc[elbow_idx, 'feature']
          elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

          plt.figure(figsize=(10, 6))
          plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
          plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
          plt.title("MLR SHAP Feature Importance with Elbow Point")
          plt.xlabel("Feature Rank")
          plt.ylabel("Mean |SHAP value|")
          plt.legend()
          plt.grid(True)
          plt.show()

          shap.summary_plot(shap_values_mlr, X_train_scaled_df, plot_type="dot", show=True)


      elif reforecast_model == 'svr':
          # ------------------- SVR -------------------
          X_sample_df, y_sample = resample(X_train_scaled_df, y_train, n_samples=5000, random_state=42)
          importance_dict = permutation_importance_per_instance(
              columns=X_sample_df.columns,
              model=reforecast_svr,
              X=X_sample_df,
              n_repeats=5,
              n_jobs=-1  # 모든 CPU 코어 사용
          )

          # ▶ 평균 중요도로 정리
          shap_style_df = pd.DataFrame({
              'feature': X_sample_df.columns,
              'mean_abs_shap': [np.mean(np.abs(v)) for v in importance_dict.values()]
          }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

          # ▶ 출력
          for _, row in shap_style_df.iterrows():
              print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

          # elbow point
          x = np.arange(len(shap_style_df))
          y = shap_style_df['mean_abs_shap'].values
          x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
          y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

          start = np.array([x_scaled[0], y_scaled[0]])
          end = np.array([x_scaled[-1], y_scaled[-1]])

          line_vec = end - start
          line_vec_norm = line_vec / np.linalg.norm(line_vec)
          point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
          projections = np.dot(point_vecs, line_vec_norm)
          projected_points = np.outer(projections, line_vec_norm) + start
          distances = np.linalg.norm(point_vecs - projected_points, axis=1)

          elbow_idx = np.argmax(distances)
          elbow_feature = shap_style_df.loc[elbow_idx, 'feature']
          elbow_value = shap_style_df.loc[elbow_idx, 'mean_abs_shap']

          plt.figure(figsize=(10, 6))
          plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
          plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
          plt.title("SVR SHAP Feature Importance with Elbow Point")
          plt.xlabel("Feature Rank")
          plt.ylabel("Mean |SHAP value|")
          plt.legend()
          plt.grid(True)
          plt.show()

          svr_array = np.stack([v for v in importance_dict.values()], axis=-1)
          shap.summary_plot(svr_array, X_sample_df, plot_type="dot", show=True)


      elif reforecast_model == 'lgb':
      # ------------------- LGB -------------------explainer_lgb = shap.Explainer(reforecast_lgb_model)

        explainer_lgb = shap.Explainer(reforecast_lgb_model)
        shap_values_lgb = explainer_lgb(X_train)

        shap_importance = np.abs(shap_values_lgb.values).mean(axis=0)

        # DataFrame으로 정리
        shap_df = pd.DataFrame({
            'feature': X_test.columns,
            'mean_abs_shap': shap_importance
        }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

        # 출력
        print("\n[LGB Feature Importance (SHAP)]")
        for _, row in shap_df.iterrows():
            print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

        # kneedle 알고리즘 사용
        x = np.arange(len(shap_df))
        y = shap_df['mean_abs_shap'].values
        x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
        y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

        # 직선 시작점과 끝점
        start = np.array([x_scaled[0], y_scaled[0]])
        end = np.array([x_scaled[-1], y_scaled[-1]])

        # 각 점에서 직선까지의 수직 거리 계산
        line_vec = end - start
        line_vec_norm = line_vec / np.linalg.norm(line_vec)
        point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
        projections = np.dot(point_vecs, line_vec_norm)
        projected_points = np.outer(projections, line_vec_norm) + start
        distances = np.linalg.norm(point_vecs - projected_points, axis=1)

        # 가장 멀리 떨어진 점이 elbow point
        elbow_idx = np.argmax(distances)
        elbow_feature = shap_df.loc[elbow_idx, 'feature']
        elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

        # 시각화
        plt.figure(figsize=(10, 6))
        plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
        plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
        plt.title("SHAP Feature Importance with Elbow Point (Distance Method)")
        plt.xlabel("Feature Rank")
        plt.ylabel("Mean |SHAP value|")
        plt.legend()
        plt.grid(True)
        plt.show()

        shap.summary_plot(shap_values_lgb, X_train, plot_type="dot", show=True)


      elif reforecast_model == 'mlp':
        # ------------------- MLP -------------------
        def compute_shap_partial(batch):
            explainer = shap.KernelExplainer(reforecast_mlp.predict, background)
            return explainer.shap_values(batch)

        X_target = shap.utils.sample(X_train_scaled_df, 2000, random_state=42)
        background = shap.utils.sample(X_target, 100, random_state=42)
        X_batches = np.array_split(X_target, 8)
        with multiprocessing.Pool(processes=8) as pool:
            shap_parts = pool.map(compute_shap_partial, X_batches)
        shap_values = np.concatenate(shap_parts, axis=0)

        shap_importance = np.abs(shap_values).mean(axis=0)
        shap_df = pd.DataFrame({
            'feature': X_train_scaled_df.columns,
            'mean_abs_shap': shap_importance
        }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

        print("\n[MLP Feature Importance (SHAP)]")
        for _, row in shap_df.iterrows():
            print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

        # elbow point 계산 및 시각화
        x = np.arange(len(shap_df))
        y = shap_df['mean_abs_shap'].values
        x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
        y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()
        start = np.array([x_scaled[0], y_scaled[0]])
        end = np.array([x_scaled[-1], y_scaled[-1]])
        line_vec = end - start
        line_vec_norm = line_vec / np.linalg.norm(line_vec)
        point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
        projections = np.dot(point_vecs, line_vec_norm)
        projected_points = np.outer(projections, line_vec_norm) + start
        distances = np.linalg.norm(point_vecs - projected_points, axis=1)
        elbow_idx = np.argmax(distances)
        elbow_feature = shap_df.loc[elbow_idx, 'feature']
        elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

        plt.figure(figsize=(10, 6))
        plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
        plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
        plt.title("MLP SHAP Feature Importance with Elbow Point")
        plt.xlabel("Feature Rank")
        plt.ylabel("Mean |SHAP value|")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        shap.summary_plot(shap_values, X_target, plot_type="dot", show=True)



논문을 위해 reforecast best 만, svr 도 shap

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import multiprocessing



forecast_models = ['mlr','svr','lgb', 'mlp']
reforecast_models = ['mlr','svr','lgb', 'mlp']

mlr_files = sorted([os.path.join(path+'/result_of_paper/ann/', file) for file in os.listdir(path+'/result_of_paper/ann/') if file.endswith('.csv') and 'mlr' in file])
svr_files = sorted([os.path.join(path+'/result_of_paper/ann/', file) for file in os.listdir(path+'/result_of_paper/ann/') if file.endswith('.csv') and 'svr' in file])
lgb_files = sorted([os.path.join(path+'/result_of_paper/ann/', file) for file in os.listdir(path+'/result_of_paper/ann/') if file.endswith('.csv') and 'lgb' in file])
mlp_files = sorted([os.path.join(path+'/result_of_paper/ann/', file) for file in os.listdir(path+'/result_of_paper/ann/') if file.endswith('.csv') and 'mlp' in file])

mlr_df_list = []
svr_df_list = []
lgb_df_list = []
mlp_df_list = []

for file in mlr_files:
  mlr_df_list.append(pd.read_csv(file))
for file in svr_files:
  svr_df_list.append(pd.read_csv(file))
for file in lgb_files:
  lgb_df_list.append(pd.read_csv(file))
for file in mlp_files:
  mlp_df_list.append(pd.read_csv(file))

best_model_tuples = [
    ('svr', 'svr'),
    ('lgb', 'svr'),
    ('mlr', 'svr'),
    ('mlr', 'svr'),
    ('mlr', 'svr'),
    ('mlr', 'svr'),
    ('lgb', 'svr')
]
test_size = 2/24

df_lists=[mlr_df_list, svr_df_list,lgb_df_list, mlp_df_list]

for forecast_model, df_list in zip(forecast_models, df_lists):
  print('==============================================================================================')
  print(f'forecast_model : {forecast_model.upper()}')

  for df, site,(best_forecast_model, best_reforecast_model) in zip(df_list, [1,2,4,5,6,7,8], best_model_tuples):

    if best_forecast_model != forecast_model:
      print(f'site : {site}, best_forecast_model : {best_forecast_model}, forecast_model = {forecast_model}')
      continue

    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print(f'site : {site}')
    target = f'error_{forecast_model}'
    shifted_target = f'{target}_shifted'

    for reforecast_model in reforecast_models:
      if best_reforecast_model != reforecast_model:
        continue

      print('--------------------------------------------------------------------------------------------------------------------------------')

      df_copy = df.copy()
      df_copy[shifted_target] = df_copy[target].shift(-1)
      df_copy = df_copy.dropna().reset_index(drop=True)

      split_index = int(len(df_copy) * (1 - test_size))

      added_features = features.copy() + [f'error_{forecast_model}', f'pred_{forecast_model}']
      added_features = [col for col in added_features if col in df_copy.columns]
      X = df_copy[added_features]
      y = df_copy[shifted_target]
      X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
      y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

      scaler = StandardScaler()
      X_train_scaled = scaler.fit_transform(X_train)
      X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
      X_test_scaled = scaler.transform(X_test)
      X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

      reforecast_mlr = joblib.load(f"{path}/result_of_paper/ann/mlr_reforecasting_model_{forecast_model}_forecast_{str(site)}.joblib")
      reforecast_svr = joblib.load(f"{path}/result_of_paper/ann/svr_reforecasting_model_{forecast_model}_forecast_{str(site)}.joblib")
      reforecast_lgb_model = lgb.Booster(model_file=f"{path}/result_of_paper/ann/lgb_reforecasting_model_{forecast_model}_forecast_{str(site)}.txt")
      reforecast_mlp = joblib.load(f"{path}/result_of_paper/ann/mlp_reforecasting_model_{forecast_model}_forecast_{site}.joblib")


      if reforecast_model == 'mlr':
          # ------------------- MLR -------------------
          explainer_mlr = shap.Explainer(reforecast_mlr, X_test_scaled_df)
          shap_values_mlr = explainer_mlr(X_test_scaled_df)

          shap_importance = np.abs(shap_values_mlr.values).mean(axis=0)

          shap_df = pd.DataFrame({
              'feature': X_train.columns,
              'mean_abs_shap': shap_importance
          }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

          print("\n[MLR Feature Importance (SHAP)]")
          for _, row in shap_df.iterrows():
            print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

          # elbow point
          x = np.arange(len(shap_df))
          y = shap_df['mean_abs_shap'].values
          x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
          y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

          start = np.array([x_scaled[0], y_scaled[0]])
          end = np.array([x_scaled[-1], y_scaled[-1]])

          line_vec = end - start
          line_vec_norm = line_vec / np.linalg.norm(line_vec)
          point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
          projections = np.dot(point_vecs, line_vec_norm)
          projected_points = np.outer(projections, line_vec_norm) + start
          distances = np.linalg.norm(point_vecs - projected_points, axis=1)

          elbow_idx = np.argmax(distances)
          elbow_feature = shap_df.loc[elbow_idx, 'feature']
          elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

          plt.figure(figsize=(10, 6))
          plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
          plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
          plt.title("MLR SHAP Feature Importance with Elbow Point")
          plt.xlabel("Feature Rank")
          plt.ylabel("Mean |SHAP value|")
          plt.legend()
          plt.grid(True)
          plt.show()

          shap.summary_plot(shap_values_mlr, X_test_scaled_df, plot_type="dot", show=True)


      elif reforecast_model == 'svr':
        X_sample_df, y_sample = resample(X_train_scaled_df, y_train, n_samples=10, random_state=42)

        # X_sample_df = X_test_scaled_df.copy()

        # ▶ SHAP: 배경 데이터 (작게 유지)
        background = shap.utils.sample(X_sample_df, 10, random_state=42)

        # ▶ SHAP 계산 (KernelExplainer 사용)
        explainer = shap.KernelExplainer(reforecast_svr.predict, background)
        shap_values = explainer.shap_values(X_sample_df)

        # ▶ 평균 중요도 계산
        mean_abs_shap = np.abs(shap_values).mean(axis=0)
        shap_style_df = pd.DataFrame({
            'feature': X_sample_df.columns,
            'mean_abs_shap': mean_abs_shap
        }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

        # ▶ 출력
        print("\n[SVR Feature Importance (SHAP)]")
        for _, row in shap_style_df.iterrows():
            print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

        # ▶ Elbow point 계산
        x = np.arange(len(shap_style_df))
        y = shap_style_df['mean_abs_shap'].values
        x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
        y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

        start = np.array([x_scaled[0], y_scaled[0]])
        end = np.array([x_scaled[-1], y_scaled[-1]])
        line_vec = end - start
        line_vec_norm = line_vec / np.linalg.norm(line_vec)
        point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
        projections = np.dot(point_vecs, line_vec_norm)
        projected_points = np.outer(projections, line_vec_norm) + start
        distances = np.linalg.norm(point_vecs - projected_points, axis=1)

        elbow_idx = np.argmax(distances)
        elbow_feature = shap_style_df.loc[elbow_idx, 'feature']
        elbow_value = shap_style_df.loc[elbow_idx, 'mean_abs_shap']

        # ▶ 시각화
        plt.figure(figsize=(10, 6))
        plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
        plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
        plt.title("SVR SHAP Feature Importance with Elbow Point")
        plt.xlabel("Feature Rank")
        plt.ylabel("Mean |SHAP value|")
        plt.legend()
        plt.grid(True)
        plt.show()

        # ▶ SHAP Summary Plot
        shap.summary_plot(shap_values, X_sample_df, plot_type="dot", show=True)


      elif reforecast_model == 'lgb':
      # ------------------- LGB -------------------explainer_lgb = shap.Explainer(reforecast_lgb_model)

        explainer_lgb = shap.Explainer(reforecast_lgb_model)
        shap_values_lgb = explainer_lgb(X_test)

        shap_importance = np.abs(shap_values_lgb.values).mean(axis=0)

        # DataFrame으로 정리
        shap_df = pd.DataFrame({
            'feature': X_test.columns,
            'mean_abs_shap': shap_importance
        }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

        # 출력
        print("\n[LGB Feature Importance (SHAP)]")
        for _, row in shap_df.iterrows():
            print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

        # kneedle 알고리즘 사용
        x = np.arange(len(shap_df))
        y = shap_df['mean_abs_shap'].values
        x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
        y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

        # 직선 시작점과 끝점
        start = np.array([x_scaled[0], y_scaled[0]])
        end = np.array([x_scaled[-1], y_scaled[-1]])

        # 각 점에서 직선까지의 수직 거리 계산
        line_vec = end - start
        line_vec_norm = line_vec / np.linalg.norm(line_vec)
        point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
        projections = np.dot(point_vecs, line_vec_norm)
        projected_points = np.outer(projections, line_vec_norm) + start
        distances = np.linalg.norm(point_vecs - projected_points, axis=1)

        # 가장 멀리 떨어진 점이 elbow point
        elbow_idx = np.argmax(distances)
        elbow_feature = shap_df.loc[elbow_idx, 'feature']
        elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

        # 시각화
        plt.figure(figsize=(10, 6))
        plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
        plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
        plt.title("SHAP Feature Importance with Elbow Point (Distance Method)")
        plt.xlabel("Feature Rank")
        plt.ylabel("Mean |SHAP value|")
        plt.legend()
        plt.grid(True)
        plt.show()

        shap.summary_plot(shap_values_lgb, X_test, plot_type="dot", show=True)


      elif reforecast_model == 'mlp':
        # ------------------- MLP -------------------
        def compute_shap_partial(batch):
            explainer = shap.KernelExplainer(reforecast_mlp.predict, background)
            return explainer.shap_values(batch)

        X_target = shap.utils.sample(X_test_scaled_df, 2000, random_state=42)
        background = shap.utils.sample(X_target, 100, random_state=42)
        X_batches = np.array_split(X_target, 8)
        with multiprocessing.Pool(processes=8) as pool:
            shap_parts = pool.map(compute_shap_partial, X_batches)
        shap_values = np.concatenate(shap_parts, axis=0)

        shap_importance = np.abs(shap_values).mean(axis=0)
        shap_df = pd.DataFrame({
            'feature': X_train_scaled_df.columns,
            'mean_abs_shap': shap_importance
        }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

        print("\n[MLP Feature Importance (SHAP)]")
        for _, row in shap_df.iterrows():
            print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

        # elbow point 계산 및 시각화
        x = np.arange(len(shap_df))
        y = shap_df['mean_abs_shap'].values
        x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
        y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()
        start = np.array([x_scaled[0], y_scaled[0]])
        end = np.array([x_scaled[-1], y_scaled[-1]])
        line_vec = end - start
        line_vec_norm = line_vec / np.linalg.norm(line_vec)
        point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
        projections = np.dot(point_vecs, line_vec_norm)
        projected_points = np.outer(projections, line_vec_norm) + start
        distances = np.linalg.norm(point_vecs - projected_points, axis=1)
        elbow_idx = np.argmax(distances)
        elbow_feature = shap_df.loc[elbow_idx, 'feature']
        elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

        plt.figure(figsize=(10, 6))
        plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
        plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
        plt.title("MLP SHAP Feature Importance with Elbow Point")
        plt.xlabel("Feature Rank")
        plt.ylabel("Mean |SHAP value|")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        shap.summary_plot(shap_values, X_target, plot_type="dot", show=True)



## Forecast test

In [ ]:
from sklearn.preprocessing import MinMaxScaler


for i, df_index in enumerate(a):
  print('----------------------------------------------------------------------------------')
  for forecast_model in ['mlr','svr','lgb','mlp']:
    start_str, end_str = operation_hours_array[i]
    file = [os.path.join(path + '/lag_added_dataset/', file)
            for file in os.listdir(path + '/lag_added_dataset/')
            if file.endswith('.csv') and f'{str(df_index)}' in file and 'feature_added_dataset_Solar' in file][0]
    print(file)
    df = pd.read_csv(file)

    target = 'Power (MW)'
    shifted_target = 'Power_shifted'
    df[shifted_target] = df[target].shift(-1)
    df = df.dropna().reset_index(drop=True)

    split_index = int(len(df) * (1 - test_size))

    added_features = [col for col in features if col in df.columns]
    X = df[added_features]
    y = df[shifted_target]
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)


    if forecast_model == 'mlr':
      model = joblib.load(f"{path}/result_of_paper/ann/mlr_forecasting_model_{df_index}.joblib")
      explainer = shap.Explainer(model, X_train_scaled_df)
      shap_values = explainer(X_test_scaled_df)

      importance_df = pd.DataFrame({
        'feature': X_test_scaled_df.columns,
        'mean_abs_shap': np.abs(shap_values.values).mean(axis=0)
      }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

    elif forecast_model == 'svr':
      model = joblib.load(f"{path}/result_of_paper/ann/svr_forecasting_model_{df_index}.joblib")

      X_sample_df, y_sample = resample(X_train_scaled_df, y_train, n_samples=5000, random_state=42)
      importance_dict = permutation_importance_per_instance(
          columns=X_sample_df.columns,
          model=model,
          X=X_sample_df,
          n_repeats=5,
          n_jobs=-1
      )

      importance_df = pd.DataFrame({
          'feature': X_sample_df.columns,
          'mean_abs_shap': [np.mean(np.abs(v)) for v in importance_dict.values()]
      }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

    elif forecast_model == 'lgb':
      model = lgb.Booster(model_file=f"{path}/result_of_paper/ann/lgb_forecasting_model_{df_index}.txt")
      explainer = shap.Explainer(model)
      shap_values = explainer(X_test)
      importance_df = pd.DataFrame({
          'feature': X_test.columns,
          'mean_abs_shap': np.abs(shap_values.values).mean(axis=0)
      }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

    elif forecast_model == 'mlp':
      model = joblib.load(f"{path}/result_of_paper/ann/mlp_forecasting_model_{df_index}.joblib")
      X_sample_df, y_sample = resample(X_train_scaled_df, y_train, n_samples=5000, random_state=42)
      importance_dict = permutation_importance_per_instance(
          columns=X_sample_df.columns,
          model=model,
          X=X_sample_df,
          n_repeats=5,
          n_jobs=-1
      )
      importance_df = pd.DataFrame({
          'feature': X_sample_df.columns,
          'mean_abs_shap': [np.mean(np.abs(v)) for v in importance_dict.values()]
      }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

    if forecast_model in ['mlr', 'svr', 'lgb', 'mlp']:
      print(f"\n[{forecast_model.upper()} Feature Importance]")
      for _, row in importance_df.iterrows():
        print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

      # Elbow point 계산
      x = np.arange(len(importance_df))
      y = importance_df['mean_abs_shap'].values
      x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
      y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

      start = np.array([x_scaled[0], y_scaled[0]])
      end = np.array([x_scaled[-1], y_scaled[-1]])
      line_vec = end - start
      line_vec_norm = line_vec / np.linalg.norm(line_vec)
      point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
      projections = np.dot(point_vecs, line_vec_norm)
      projected_points = np.outer(projections, line_vec_norm) + start
      distances = np.linalg.norm(point_vecs - projected_points, axis=1)

      elbow_idx = np.argmax(distances)
      elbow_feature = importance_df.loc[elbow_idx, 'feature']
      elbow_value = importance_df.loc[elbow_idx, 'mean_abs_shap']

      # 시각화
      plt.figure(figsize=(10, 6))
      plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
      plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
      plt.title(f"{forecast_model.upper()} SHAP Feature Importance")
      plt.xlabel("Feature Rank")
      plt.ylabel("Mean |SHAP value|")
      plt.legend()
      plt.grid(True)
      plt.show()


## Forecast train

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import multiprocessing


forecast_models = ['mlr','svr','lgb', 'mlp']

files = sorted([os.path.join(path+'/result_of_paper/ann/', file) for file in os.listdir(path+'/result_of_paper/ann/') if file.endswith('.csv') and 'mlr' in file])

df_list = []

for file in files:
  df_list.append(pd.read_csv(file))

test_size = 2/24

for df, site in zip(df_list, [1,2,4,5,6,7,8]):
  print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
  print(f'site : {site}')
  target = 'Power (MW)'
  shifted_target = 'power_shifted'

  df_copy = df.copy()
  df_copy[shifted_target] = df_copy[target].shift(-1)
  df_copy = df_copy.dropna().reset_index(drop=True)

  split_index = int(len(df_copy) * (1 - test_size))

  X = df_copy[features]
  y = df_copy[shifted_target]
  X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
  y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
  X_test_scaled = scaler.transform(X_test)
  X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

  forecast_mlr = joblib.load(f"{path}/result_of_paper/ann/mlr_forecasting_model_{str(site)}.joblib")
  forecast_svr = joblib.load(f"{path}/result_of_paper/ann/svr_forecasting_model_{str(site)}.joblib")
  forecast_lgb_model = lgb.Booster(model_file=f"{path}/result_of_paper/ann/lgb_forecasting_model_{str(site)}.txt")
  forecast_mlp = joblib.load(f"{path}/result_of_paper/ann/mlp_forecasting_model_{str(site)}.joblib")

  for forecast_model in ['mlr','svr','lgb','mlp']:
    print('--------------------------------------------------------------------------------------------------------------------------------')
    print(f'forecast_model : {forecast_model}')

    if forecast_model == 'mlr':
        # ------------------- MLR -------------------
        explainer_mlr = shap.Explainer(forecast_mlr, X_train_scaled_df)
        shap_values_mlr = explainer_mlr(X_train_scaled_df)

        shap_importance = np.abs(shap_values_mlr.values).mean(axis=0)

        shap_df = pd.DataFrame({
            'feature': X_train.columns,
            'mean_abs_shap': shap_importance
        }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

        print("\n[MLR Feature Importance (SHAP)]")
        for _, row in shap_df.iterrows():
          print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

        # elbow point
        x = np.arange(len(shap_df))
        y = shap_df['mean_abs_shap'].values
        x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
        y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

        start = np.array([x_scaled[0], y_scaled[0]])
        end = np.array([x_scaled[-1], y_scaled[-1]])

        line_vec = end - start
        line_vec_norm = line_vec / np.linalg.norm(line_vec)
        point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
        projections = np.dot(point_vecs, line_vec_norm)
        projected_points = np.outer(projections, line_vec_norm) + start
        distances = np.linalg.norm(point_vecs - projected_points, axis=1)

        elbow_idx = np.argmax(distances)
        elbow_feature = shap_df.loc[elbow_idx, 'feature']
        elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

        plt.figure(figsize=(10, 6))
        plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
        plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
        plt.title("MLR SHAP Feature Importance with Elbow Point")
        plt.xlabel("Feature Rank")
        plt.ylabel("Mean |SHAP value|")
        plt.legend()
        plt.grid(True)
        plt.show()

    elif forecast_model == 'svr':
        # ------------------- SVR -------------------
        X_sample_df, y_sample = resample(X_train_scaled_df, y_train, n_samples=5000, random_state=42)
        importance_dict = permutation_importance_per_instance(
            columns=X_sample_df.columns,
            model=forecast_svr,
            X=X_sample_df,
            n_repeats=5,
            n_jobs=-1  # 모든 CPU 코어 사용
        )

        # ▶ 평균 중요도로 정리
        shap_style_df = pd.DataFrame({
            'feature': X_sample_df.columns,
            'mean_abs_shap': [np.mean(np.abs(v)) for v in importance_dict.values()]
        }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

        # ▶ 출력
        for _, row in shap_style_df.iterrows():
            print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

        # elbow point
        x = np.arange(len(shap_style_df))
        y = shap_style_df['mean_abs_shap'].values
        x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
        y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

        start = np.array([x_scaled[0], y_scaled[0]])
        end = np.array([x_scaled[-1], y_scaled[-1]])

        line_vec = end - start
        line_vec_norm = line_vec / np.linalg.norm(line_vec)
        point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
        projections = np.dot(point_vecs, line_vec_norm)
        projected_points = np.outer(projections, line_vec_norm) + start
        distances = np.linalg.norm(point_vecs - projected_points, axis=1)

        elbow_idx = np.argmax(distances)
        elbow_feature = shap_style_df.loc[elbow_idx, 'feature']
        elbow_value = shap_style_df.loc[elbow_idx, 'mean_abs_shap']

        plt.figure(figsize=(10, 6))
        plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
        plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
        plt.title("SVR SHAP Feature Importance with Elbow Point")
        plt.xlabel("Feature Rank")
        plt.ylabel("Mean |SHAP value|")
        plt.legend()
        plt.grid(True)
        plt.show()


    elif forecast_model == 'lgb':
    # ------------------- LGB -------------------explainer_lgb = shap.Explainer(reforecast_lgb_model)

      explainer_lgb = shap.Explainer(forecast_lgb_model)
      shap_values_lgb = explainer_lgb(X_train)

      shap_importance = np.abs(shap_values_lgb.values).mean(axis=0)

      # DataFrame으로 정리
      shap_df = pd.DataFrame({
          'feature': X_test.columns,
          'mean_abs_shap': shap_importance
      }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

      # 출력
      print("\n[LGB Feature Importance (SHAP)]")
      for _, row in shap_df.iterrows():
          print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

      # kneedle 알고리즘 사용
      x = np.arange(len(shap_df))
      y = shap_df['mean_abs_shap'].values
      x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
      y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

      # 직선 시작점과 끝점
      start = np.array([x_scaled[0], y_scaled[0]])
      end = np.array([x_scaled[-1], y_scaled[-1]])

      # 각 점에서 직선까지의 수직 거리 계산
      line_vec = end - start
      line_vec_norm = line_vec / np.linalg.norm(line_vec)
      point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
      projections = np.dot(point_vecs, line_vec_norm)
      projected_points = np.outer(projections, line_vec_norm) + start
      distances = np.linalg.norm(point_vecs - projected_points, axis=1)

      # 가장 멀리 떨어진 점이 elbow point
      elbow_idx = np.argmax(distances)
      elbow_feature = shap_df.loc[elbow_idx, 'feature']
      elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

      # 시각화
      plt.figure(figsize=(10, 6))
      plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
      plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
      plt.title("SHAP Feature Importance with Elbow Point (Distance Method)")
      plt.xlabel("Feature Rank")
      plt.ylabel("Mean |SHAP value|")
      plt.legend()
      plt.grid(True)
      plt.show()

    elif forecast_model == 'mlp':
      # ------------------- MLP -------------------
      def compute_shap_partial(batch):
          explainer = shap.KernelExplainer(forecast_mlp.predict, background)
          return explainer.shap_values(batch)

      X_target = shap.utils.sample(X_train_scaled_df, 2000, random_state=42)
      background = shap.utils.sample(X_target, 100, random_state=42)
      X_batches = np.array_split(X_target, 8)
      with multiprocessing.Pool(processes=8) as pool:
          shap_parts = pool.map(compute_shap_partial, X_batches)
      shap_values_mlp = np.concatenate(shap_parts, axis=0)

      shap_importance = np.abs(shap_values_mlp).mean(axis=0)
      shap_df = pd.DataFrame({
          'feature': X_train_scaled_df.columns,
          'mean_abs_shap': shap_importance
      }).sort_values('mean_abs_shap', ascending=False).reset_index(drop=True)

      print("\n[MLP Feature Importance (SHAP)]")
      for _, row in shap_df.iterrows():
          print(f"{row['feature']}: {row['mean_abs_shap']:.6f}")

      x = np.arange(len(shap_df))
      y = shap_df['mean_abs_shap'].values
      x_scaled = MinMaxScaler().fit_transform(x.reshape(-1, 1)).flatten()
      y_scaled = MinMaxScaler().fit_transform(y.reshape(-1, 1)).flatten()

      start = np.array([x_scaled[0], y_scaled[0]])
      end = np.array([x_scaled[-1], y_scaled[-1]])
      line_vec = end - start
      line_vec_norm = line_vec / np.linalg.norm(line_vec)
      point_vecs = np.stack([x_scaled - start[0], y_scaled - start[1]], axis=1)
      projections = np.dot(point_vecs, line_vec_norm)
      projected_points = np.outer(projections, line_vec_norm) + start
      distances = np.linalg.norm(point_vecs - projected_points, axis=1)

      elbow_idx = np.argmax(distances)
      elbow_feature = shap_df.loc[elbow_idx, 'feature']
      elbow_value = shap_df.loc[elbow_idx, 'mean_abs_shap']

      plt.figure(figsize=(10, 6))
      plt.plot(x, y, marker='o', label='SHAP importance', linewidth=2)
      plt.scatter(elbow_idx, elbow_value, color='red', marker='*', s=300, label=f'Elbow Point: {elbow_feature}')
      plt.title("MLP SHAP Feature Importance with Elbow Point")
      plt.xlabel("Feature Rank")
      plt.ylabel("Mean |SHAP value|")
      plt.legend()
      plt.grid(True)
      plt.show()
